<a href="https://colab.research.google.com/github/xAngel563x/ABP/blob/master/AnalisisSentimiento.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

import pandas as pd

trainingData = pd.read_csv('/content/marvel-train.csv')
trainingData = trainingData.head(1000) #Eliminar la funcion head() si se quiere usar todo el dataset. Para las pruebas usamos únicamente los 1000 primeros tweets
trainingData

,Unnamed: 0,comic_name,active_years,issue_title,publish_date,issue_description,penciler,writer,cover_artist,Imprint,Format,Rating,Price,label,text
0,0,A Year of Marvels: June Infinite Comic (2016),(2016),A Year of Marvels: June Infinite Comic (2016) #1,"June 15, 2016",Sam Alexander’s finding it hard to cope with t...,Diego Olortegui,Paul Allor,Jamal Campbell,Marvel Universe,Infinite Comic,NaN,Free,1,Cap knows the future. When something like the ...
1,1,A+X (2012 - 2014),(2012 - 2014),A+X (2012) #17,"February 19, 2014",SPIDER-MAN + PSYLOCKE= a heart-breaking story ...,"Paco Diaz, David Yardin","Gerry Duggan, Jeff Loveness",Goran Parlov,NaN,Comic,Rated T,$3.99,1,When they were explaining the Time Travel rule...
2,2,A+X (2012 - 2014),(2012 - 2014),A+X (2012) #15,"December 11, 2013",Dr. Strange + Beast = a truly insane story by ...,Greg Smallwood,"Jai Nitz, Gerry Duggan",NaN,NaN,Comic,NaN,$3.99,-1,Natasha's death was the culmination of what we...
3,3,A+X (2012 - 2014),(2012 - 2014),A+X (2012) #14,"November 20, 2013",CAPTAIN AMERICA + CYCLOPS continue their quest...,David Lafuente,Gerry Duggan,NaN,NaN,Comic,NaN,$3.99,1,No one said this yet so I guess it's an (extre...
4,4,A+X (2012 - 2014),(2012 - 2014),A+X (2012) #8,"May 22, 2013",Captain America and Wolverine fight a villain ...,Humberto Ramos,Nathan Edmondson,NaN,NaN,Comic,NaN,$3.99,-1,"I'm saying it's not the biggest movie, inflati..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,995,Dazzler (1981 - 1986),(1981 - 1986),Dazzler (1981) #34,"October 10, 1984",NaN,Geof Isherwood,Mike Carlin,NaN,NaN,Comic,NaN,Free,0,How did Nebula know there would be a sacrifice...
996,996,Dazzler (1981 - 1986),(1981 - 1986),Dazzler (1981) #12,"February 01, 1982",All Dazzler wants is a job! And when she's off...,Frank Springer,Danny Fingeroth,NaN,NaN,Comic,NaN,Free,1,I honestly thought that when Cap returned he'd...
997,997,Dazzler Facsimile Edition (2019),(2019),Dazzler Facsimile Edition (2019) #1,"February 20, 2019","She’s a singer, not a fighter — but Dazzler wa...","John Romita, John Buscema",Tom Defalco,Bob Larkin,Marvel Universe,Comic,Rated T,$3.99,1,I was hoping it would be Bucky who would becom...
998,998,Dead Man Logan (2018 - 2019),(2018 - 2019),Dead Man Logan (2018) #10,"August 07, 2019",TEN DEGREES OF EVISCERATION! Logan's oldest fo...,Mike Henderson,Ed Brisson,Declan Shalvey,NaN,Comic,Parental Advisory,$3.99,1,"Only thing I can answer for you is, Thor drank..."


Preprocesamiento de los datos de entrenamiento


In [ ]:
trainingData['label'].value_counts()

,count
label,
1,480
-1,322
0,198


In [ ]:

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

import nltk
nltk.download('punkt_tab')
nltk.download('stopwords')


import os
print(os.path.exists(nltk.data.find('tokenizers/punkt')))

ps = PorterStemmer()

preprocessedText = []

for row in trainingData.itertuples():


    text = word_tokenize(row.text) ## indice de la columna que contiene el texto
    ## Remove stop words
    stops = set(stopwords.words("english"))
    text = [ps.stem(w) for w in text if not w in stops and w.isalnum()]
    text = " ".join(text)

    preprocessedText.append(text)

preprocessedData = trainingData
preprocessedData['processed_text'] = preprocessedText

preprocessedData


[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True


,label,text,processed_text
0,1,This is a sample sentence.,thi sampl sentenc
1,0,Another example sentence!,anoth exampl sentenc


Creación de la bolsa de palabras


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

bagOfWordsModel = TfidfVectorizer()
bagOfWordsModel.fit(preprocessedData['processed_text'])
textsBoW= bagOfWordsModel.transform(preprocessedData['processed_text'])
print("Finished")

Finished


In [ ]:
textsBoW.shape


(2, 5)

Entrenamiento de un algoritmo de clasificación (SVM)


In [ ]:
from sklearn import svm
svc = svm.SVC(kernel='linear') #Modelo de clasificación

X_train = textsBoW #Documentos
Y_train = trainingData['label'] #Etiquetas de los documentos
svc.fit(X_train, Y_train) #Entrenamiento

SVC(kernel='linear')

Carga de y preprocesado de documentos de test


In [ ]:
testData = pd.read_csv('/content/marvel-test3.csv')
testData = testData.head(100)
testData

,Unnamed: 0,comic_name,active_years,issue_title,publish_date,issue_description,penciler,writer,cover_artist,Imprint,Format,Rating,Price,label,text
0,0,A Year of Marvels: June Infinite Comic (2016),(2016),A Year of Marvels: June Infinite Comic (2016) #1,"June 15, 2016",Sam Alexander’s finding it hard to cope with t...,Diego Olortegui,Paul Allor,Jamal Campbell,Marvel Universe,Infinite Comic,NaN,Free,1,Cap knows the future. When something like the ...
1,1,A+X (2012 - 2014),(2012 - 2014),A+X (2012) #17,"February 19, 2014",SPIDER-MAN + PSYLOCKE= a heart-breaking story ...,"Paco Diaz, David Yardin","Gerry Duggan, Jeff Loveness",Goran Parlov,NaN,Comic,Rated T,$3.99,1,When they were explaining the Time Travel rule...
2,2,A+X (2012 - 2014),(2012 - 2014),A+X (2012) #15,"December 11, 2013",Dr. Strange + Beast = a truly insane story by ...,Greg Smallwood,"Jai Nitz, Gerry Duggan",NaN,NaN,Comic,NaN,$3.99,-1,Natasha's death was the culmination of what we...
3,3,A+X (2012 - 2014),(2012 - 2014),A+X (2012) #14,"November 20, 2013",CAPTAIN AMERICA + CYCLOPS continue their quest...,David Lafuente,Gerry Duggan,NaN,NaN,Comic,NaN,$3.99,1,No one said this yet so I guess it's an (extre...
4,4,A+X (2012 - 2014),(2012 - 2014),A+X (2012) #8,"May 22, 2013",Captain America and Wolverine fight a villain ...,Humberto Ramos,Nathan Edmondson,NaN,NaN,Comic,NaN,$3.99,-1,"I'm saying it's not the biggest movie, inflati..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,95,All-Winners Comics (1941 - 1947),(1941 - 1947),MARVEL MASTERWORKS: GOLDEN AGE ALL-WINNERS VOL...,"December 27, 2006",See the golden age in its glory as Captain Ame...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,Because he didn't age while he was snapped.
96,96,All-Winners Comics (1941 - 1947),(1941 - 1947),MARVEL MASTERWORKS: GOLDEN AGE ALL WINNERS COM...,"January 01, 2005",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,I dont think it works as a wish. He probably w...
97,97,All-Winners Comics (1941 - 1947),(1941 - 1947),All-Winners Comics (1941) #14,"November 01, 1944",The Flaming Duo investigate the death of a gir...,NaN,NaN,NaN,NaN,Comic,NaN,Free,1,This actually got me really excited for guardi...
98,98,All-Winners Comics (1941 - 1947),(1941 - 1947),All-Winners Comics (1941) #13,"September 01, 1944",The Nazis and Red Skull viciously attack capta...,NaN,NaN,NaN,NaN,Comic,NaN,Free,0,Because of all the unrealistic things to take ...


In [ ]:
ps = PorterStemmer()

preprocessedText = []

for row in testData.itertuples():


    text = word_tokenize(row.text) ## indice de la columna que contiene el texto
    ## Remove stop words
    stops = set(stopwords.words("english"))
    text = [ps.stem(w) for w in text if not w in stops and w.isalnum()]
    text = " ".join(text)

    preprocessedText.append(text)

preprocessedDataTest = testData
preprocessedDataTest['processed_text'] = preprocessedText

preprocessedDataTest

,Unnamed: 0,comic_name,active_years,issue_title,publish_date,issue_description,penciler,writer,cover_artist,Imprint,Format,Rating,Price,label,text,processed_text
0,0,A Year of Marvels: June Infinite Comic (2016),(2016),A Year of Marvels: June Infinite Comic (2016) #1,"June 15, 2016",Sam Alexander’s finding it hard to cope with t...,Diego Olortegui,Paul Allor,Jamal Campbell,Marvel Universe,Infinite Comic,NaN,Free,1,Cap knows the future. When something like the ...,cap know futur when someth like battl new york...
1,1,A+X (2012 - 2014),(2012 - 2014),A+X (2012) #17,"February 19, 2014",SPIDER-MAN + PSYLOCKE= a heart-breaking story ...,"Paco Diaz, David Yardin","Gerry Duggan, Jeff Loveness",Goran Parlov,NaN,Comic,Rated T,$3.99,1,When they were explaining the Time Travel rule...,when explain time travel rule everybodi throw ...
2,2,A+X (2012 - 2014),(2012 - 2014),A+X (2012) #15,"December 11, 2013",Dr. Strange + Beast = a truly insane story by ...,Greg Smallwood,"Jai Nitz, Gerry Duggan",NaN,NaN,Comic,NaN,$3.99,-1,Natasha's death was the culmination of what we...,natasha death culmin saw first aveng movi fina...
3,3,A+X (2012 - 2014),(2012 - 2014),A+X (2012) #14,"November 20, 2013",CAPTAIN AMERICA + CYCLOPS continue their quest...,David Lafuente,Gerry Duggan,NaN,NaN,Comic,NaN,$3.99,1,No one said this yet so I guess it's an (extre...,no one said yet i guess extrem unpopular opini...
4,4,A+X (2012 - 2014),(2012 - 2014),A+X (2012) #8,"May 22, 2013",Captain America and Wolverine fight a villain ...,Humberto Ramos,Nathan Edmondson,NaN,NaN,Comic,NaN,$3.99,-1,"I'm saying it's not the biggest movie, inflati...",i say biggest movi inflat matter if movi sold ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,95,All-Winners Comics (1941 - 1947),(1941 - 1947),MARVEL MASTERWORKS: GOLDEN AGE ALL-WINNERS VOL...,"December 27, 2006",See the golden age in its glory as Captain Ame...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,Because he didn't age while he was snapped.,becaus age snap
96,96,All-Winners Comics (1941 - 1947),(1941 - 1947),MARVEL MASTERWORKS: GOLDEN AGE ALL WINNERS COM...,"January 01, 2005",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,I dont think it works as a wish. He probably w...,i dont think work wish he probabl given capac ...
97,97,All-Winners Comics (1941 - 1947),(1941 - 1947),All-Winners Comics (1941) #14,"November 01, 1944",The Flaming Duo investigate the death of a gir...,NaN,NaN,NaN,NaN,Comic,NaN,Free,1,This actually got me really excited for guardi...,thi actual got realli excit guardian 3 thor jo...
98,98,All-Winners Comics (1941 - 1947),(1941 - 1947),All-Winners Comics (1941) #13,"September 01, 1944",The Nazis and Red Skull viciously attack capta...,NaN,NaN,NaN,NaN,Comic,NaN,Free,0,Because of all the unrealistic things to take ...,becaus unrealist thing take issu would


In [ ]:
testData['label'].value_counts()


,count
label,
1,43
-1,35
0,22


In [ ]:
textsBoWTest= bagOfWordsModel.transform(preprocessedDataTest['processed_text'])
print("Finished")

Finished


In [ ]:
textsBoWTest.shape


(100, 5)

Clasificación de los documentos de test


In [ ]:
X_test = textsBoWTest #Documentos


predictions = svc.predict(X_test) #Se almacena en el array predictions las predicciones del clasificador

Evaluación de la predicción


In [ ]:
from sklearn.metrics import classification_report

Y_test = testData['label'] #Etiquetas reales de los documentos

print (classification_report(Y_test, predictions))

              precision    recall  f1-score   support

          -1       0.00      0.00      0.00        35
           0       0.00      0.00      0.00        22
           1       0.44      0.95      0.60        43

    accuracy                           0.41       100
   macro avg       0.15      0.32      0.20       100
weighted avg       0.19      0.41      0.26       100



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Entrenamiento y Evaluación de otro algoritmo de clasificación: k-NN


In [ ]:
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=2)


neigh.fit(X_train, Y_train)
predictions = neigh.predict(X_test)

print (classification_report(Y_test, predictions))

              precision    recall  f1-score   support

          -1       0.00      0.00      0.00        35
           0       0.22      1.00      0.36        22
           1       0.00      0.00      0.00        43

    accuracy                           0.22       100
   macro avg       0.07      0.33      0.12       100
weighted avg       0.05      0.22      0.08       100



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
